In [3]:
import os
import sys
import glob
from os import write
from unittest.mock import inplace

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name, get_measure_plot_num

In [4]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s_full = idx_s_multicam + idx_s_singlecam

In [5]:
reduced_analysis = True # If True, only S001 and S002 from Multicam will be used.
if reduced_analysis:
    file_app = '_reduced'
else:
    file_app = ''
    

## kinematic measures

In [6]:
df_corr_murphy = pd.read_csv(os.path.join(dir_stat_cat, f'stat_murphy_corr{file_app}.csv'), sep=';')
df_corr_murphy['id_s_name'] = df_corr_murphy['id_s'].apply(lambda x: get_setting_axis_name(x))

if reduced_analysis:
    df_corr_murphy = df_corr_murphy[df_corr_murphy['id_s'].isin(idx_s_reduced)]

if 'Unnamed: 0' in df_corr_murphy.columns:
    df_corr_murphy = df_corr_murphy.drop(columns=['Unnamed: 0'])

In [10]:
len_before = len(df_corr_murphy)
df_corr_murphy.dropna(inplace=True)
len_after = len(df_corr_murphy)
print(f'Number of rows dropped: {len_before - len_after}')

Number of rows dropped: 0


In [7]:
df_grouped = df_corr_murphy.groupby(['id_s']).mean(numeric_only=True)

df_corr_murphy_s = pd.DataFrame(columns = ['id_s', 'id_s_name', 'correlation', 'p_value'])

for index in df_grouped.index:
    id_s = index
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)

    df_corr_murphy_s = pd.concat([df_corr_murphy_s, pd.DataFrame([[id_s, id_s_name, r, p]], columns = ['id_s', 'id_s_name', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [8]:
df_grouped = df_corr_murphy.groupby(['id_s', 'measure']).mean(numeric_only=True)

df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'measure', 'correlation', 'p_value'])

for index in df_grouped.index:
    id_s = index[0]
    measure = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, measure, r, p]], columns = ['id_s', 'id_s_name', 'measure', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [15]:
fig = px.bar(df_plot, x='id_s_name', y='correlation', color='measure', title=f'<b>Pearson Correlation<b>', labels={'correlation': 'Correlation [r] Correlation', 'Settings': 'Subject ID', 'measure': 'Kin. Measure'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
#fig.add_hline(y=-0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'<b>Pearson Correlation of kinematic measures<b>',
          xaxis_title=f'<b>Settings<b>',
          yaxis_title=f'<b>Correlation [r]<b>',
                  width=1300, height=600
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_all.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_all.png')
fig.write_image(path, scale = 5)

In [14]:
fig = px.bar(df_plot[df_plot['id_s'].isin(idx_s_reduced)], x='id_s_name', y='correlation', color='measure', title=f'Pearson Correlation', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= dict(text=f'<b>Pearson Correlation of multi- and single-cam settings<b>', font=dict(size=24)) ,
                  xaxis_title = dict(text=f'<b>Settings<b>', font=dict(size=20)),
                  yaxis_title = dict(text=f'<b>Correlation [r]<b>', font=dict(size=20)),
                  xaxis = dict(tickfont=dict(size=20)),
                  width = 1300, height = 600,
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_reduced.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_reduced.png')
fig.write_image(path, scale = 5)

In [13]:
measures = df_plot['measure'].unique()

for measure in measures:
    df_plot_measure = df_plot[df_plot['measure'] == measure]
    title_name = get_title_measure_name(measure)
    
    fig = px.bar(df_plot_measure[df_plot_measure['id_s'].isin(idx_s_reduced)], x='id_s_name', y='correlation', title=f'Pearson Correlation of {title_name}', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
                 template = 'plotly')
    
    fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
    
    fig.update_layout(title= dict(text=f'<b>Pearson Correlation of {title_name}<b>', font=dict(size=24)),
                      xaxis_title=dict(text=f'<b>Settings<b>', font=dict(size=20)),
                      yaxis_title=dict(text=f'<b>Correlation [r]<b>', font=dict(size=20)),
                      width=1000, height=500,
                      xaxis = dict(tickfont=dict(size=18))
              )
    
    fig.show()
    os.makedirs(os.path.join(dir_plots_cat, '04_correlation'), exist_ok=True)
    
    path = os.path.join(dir_plots_cat, '04_correlation', f'pearson_corr_{measure}.html')
    fig.write_html(path)
    
    suff = get_measure_plot_num(measure)
    
    dir_png = os.path.join(dir_plots_cat, '04_correlation', 'png')
    os.makedirs(dir_png, exist_ok=True)
    path = os.path.join(dir_png, f'{suff}pearson_corr_{measure}.png')
    fig.write_image(path, scale = 5)

## Timeseries


In [15]:
def restructure_dataframe(df):
    # Define the metrics
    metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

    # Initialize a list to collect rows
    rows = []

    # Iterate through each row in the original DataFrame
    for _, row in df.iterrows():
        for metric in metrics:
            rows.append({
                'id_s': row['id_s'],
                'id_p': row['id_p'],
                'id_t': row['id_t'],
                'dynamic': row['dynamic'],
                'metric': metric,
                'pearson': row[f'{metric}_pearson'],
                'pearson_p': row[f'{metric}_pearson_p']
            })

    # Create a new DataFrame from the collected rows
    new_df = pd.DataFrame(rows)
    return new_df

In [16]:
csv_corr_ts = os.path.join(dir_stat_cont, '01_results', 'time_series_correlation.csv')
csv_corr_ts = os.path.join(dir_stat_cont, '01_results', '09_ fancy_offset', 'time_series_correlation.csv')
df_corr_ts = pd.read_csv(csv_corr_ts, sep=';')

metrics_ts = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

In [17]:
'metric' not in df_corr_ts.columns

True

In [18]:
if 'metric' not in df_corr_ts.columns:
    df_corr_ts = restructure_dataframe(df_corr_ts)

In [19]:
df_corr_ts = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['metric'].isin(metrics_ts)) & (~df_corr_ts['id_p'].isin(ignore_id_p))]

In [20]:
df_corr_ts.columns

Index(['id_s', 'id_p', 'id_t', 'dynamic', 'metric', 'pearson', 'pearson_p'], dtype='object')

In [21]:
df_corr_ts_s = pd.DataFrame(columns=['id_s', 'id_s_name', 'correlation', 'p_value'])
df_grouped = df_corr_ts[df_corr_ts['id_s'].isin(idx_s_full)].groupby(['id_s']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)

    df_corr_ts_s = pd.concat([df_corr_ts_s, pd.DataFrame([[id_s, id_s_name, r, p]],
                                                    columns=['id_s', 'id_s_name', 'correlation', 'p_value'],
                                                    index=[0])], ignore_index=True)
df_corr_ts_s.sort_values(by='id_s', inplace=True)

### All Settings Joint Measures and Time-series

In [22]:
merged = pd.concat([df_corr_murphy_s, df_corr_ts_s])

df_corr_murphy_and_ts = merged.groupby(['id_s', 'id_s_name'], as_index=False).mean(numeric_only=True)

In [23]:
df_corr_murphy_and_ts.sort_values(by='correlation', ascending=False)

,id_s,id_s_name,correlation,p_value
1,S002,"Metrabs, Cams: 1,2,3,4,5",0.789126,0.004238
16,S017,"Single, Cam: 1, filt",0.719419,0.003615
0,S001,"SimCC, Cams: 1,2,3,4,5",0.687276,0.033470
7,S008,"Metrabs, Cams: 2,3,4",0.670515,0.040703
17,S018,"Single, Cam: 1, unfilt",0.661055,0.004213
12,S013,"SimCC, Cams: 2,4",0.496257,0.042706
13,S014,"Metrabs, Cams: 2,4",0.474347,0.078661
5,S006,"Metrabs, Cams: 1,3,5",0.388474,0.053147
6,S007,"SimCC, Cams: 2,3,4",0.380477,0.072564
4,S005,"SimCC, Cams: 1,3,5",0.261180,0.066029


In [24]:
fig = px.bar(df_corr_murphy_and_ts.sort_values(by='correlation', ascending=False), x='id_s_name', y='correlation', template = 'plotly')

#fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for all settings',
                  yaxis_title=f'Mean Correlation of Time-Series and Measures [r]',
                    width=1300, height=600
            )

fig.show()

os.makedirs(os.path.join(dir_plots_cont, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_joint_mean.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_joint_mean.png')
fig.write_image(path, scale = 5)

### Multi-Cam

Let's first look at all Multi-cam Settings

In [25]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'ml', 'correlation', 'p_value'])
df_grouped = df_corr_ts[df_corr_ts['dynamic'] == 'fixed'].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:        
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    ml = id_s_name.split(',')[0]
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, id_p, ml,  r, p]], columns = ['id_s', 'id_s_name', 'id_p', 'ml', 'correlation', 'p_value'], index = [0])], ignore_index=True)

df_plot = df_plot[df_plot['id_s'].isin(idx_s_multicam)]

In [26]:
fig = px.bar(df_plot.sort_values(by='id_s'), x='id_s_name', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for multi-cam settings',
          xaxis_title=f'Setting',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

os.makedirs(os.path.join(dir_plots_cont, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam.png')
fig.write_image(path, scale = 5)
fig.show()

In [27]:

df_grouped = df_plot.groupby(['id_p', 'ml']).mean(numeric_only=True)
df_plot = pd.DataFrame(columns = ['id_p', 'ml', 'correlation', 'p_value'])

for index in df_grouped.index:        
    id_p = index[0]
    ml = index[1]
    r = df_grouped.loc[index]['correlation']
    p = df_grouped.loc[index]['p_value']

    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_p, ml,  r, p]], columns = ['id_p', 'ml', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [28]:
fig = px.bar(df_plot, x= 'ml', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient", "ml": "Pose Estimation Model", 'correlation': 'Correlation [r]'})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for multi-cam settings',
                  width=1300, height=600
          )

os.makedirs(os.path.join(dir_plots_cont, '04_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam_model.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_multi_cam_model.png')
fig.write_image(path, scale = 5)
fig.show()


Settings using less than five cameras show bad performance.
### SingleCam

In [29]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_singlecam))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, id_p, r, p]], columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [30]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for single-cam settings',
                  xaxis_title=f'Setting',
                  yaxis_title=f'Correlation [r]',
                  width=1300, height=600
                  )

path = os.path.join(dir_plots_cont, '04_correlation', 'pearson_corr_single_cam.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', 'pearson_corr_single_cam.png')
fig.write_image(path)


### All Settings

In [31]:
idx_s_to_use = idx_s_reduced if reduced_analysis else idx_s_full
idx_p_to_use = [p for p in df_corr_ts['id_p'].unique() if p not in ignore_id_p]

In [32]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_to_use)) & (df_corr_ts['id_p'].isin(idx_p_to_use))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, id_p, r, p]], columns = ['id_s', 'id_s_name',  'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [33]:
fig = px.bar(df_plot.sort_values(by='id_s'), x='id_s_name', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Mean Pearson Correlation for settings and patients',
                  xaxis_title=f'Setting',
                  yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_all_settings{file_app}.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_all_settings{file_app}.png')
fig.write_image(path, scale = 5)


In [34]:
df_corr_ts

,id_s,id_p,id_t,dynamic,metric,pearson,pearson_p
0,S023,P07,T011,fixed,hand_vel,0.937507,1.791011e-241
1,S023,P07,T011,fixed,elbow_vel,0.968818,1.784763e-318
2,S023,P07,T011,fixed,trunk_disp,0.912600,6.622425e-205
3,S023,P07,T011,fixed,elbow_flex_pos,0.991970,0.000000e+00
4,S023,P07,T011,fixed,shoulder_flex_pos,0.976410,0.000000e+00
...,...,...,...,...,...,...,...
54811,S001,P252,T048,fixed,elbow_vel,0.957496,8.581049e-288
54812,S001,P252,T048,fixed,trunk_disp,0.994917,0.000000e+00
54813,S001,P252,T048,fixed,elbow_flex_pos,0.993108,0.000000e+00
54814,S001,P252,T048,fixed,shoulder_flex_pos,0.963977,2.025946e-306


In [35]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_s_name', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_to_use)) & (df_corr_ts['id_p'].isin(idx_p_to_use))].groupby(['id_s', 'metric']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    metric = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    id_s_name = get_setting_axis_name(id_s)
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_s_name, metric, r, p]], columns = ['id_s', 'id_s_name', 'metric', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

df_plot['metric'] = df_plot['metric'].apply(lambda x: get_title_measure_name(x, add_unit = False))

In [36]:
fig = px.bar(df_plot.sort_values(by='id_s'), x='id_s_name', y='correlation', color='metric', template = 'plotly', barmode='group', labels={"metric": "Kin. metrics"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= dict(text=f'<b>Pearson Correlation for chosen settings and kinematic metrics<b>', font=dict(size=20)),
                  yaxis_title=dict(text=f'<b>Mean Correlation [r]<b>', font=dict(size=18)),
                  xaxis_title=dict(text='<b>Settings<b>', font=dict(size=18)),
                  xaxis = dict(tickfont=dict(size=16)),
                  width=1000, height=500
                  )

path = os.path.join(dir_plots_cont, '04_correlation', f'pearson_corr_kinematics{file_app}.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '04_correlation', f'0307_pearson_corr_kinematics{file_app}.png')
fig.write_image(path, scale = 5)

In [37]:
fig.show()

In [38]:
df_plot.drop(columns=['id_s', 'id_p'], inplace=True)


In [39]:
c_titles = ['id_s_name', 'metric', 'correlation', 'p_value']
df_plot = df_plot.reindex(columns = c_titles)

df_plot['metric'] = df_plot['metric'].apply(lambda x: get_title_measure_name(x, add_unit = True))

In [40]:
df_plot.round(2)

,id_s_name,metric,correlation,p_value
0,"SimCC, Cams: 1,2,3,4,5",Elbow Flexion [deg],0.89,0.05
1,"SimCC, Cams: 1,2,3,4,5",Elbow Velocity [deg/s],0.78,0.01
2,"SimCC, Cams: 1,2,3,4,5",Hand Velocity [mm/s],0.78,0.02
3,"SimCC, Cams: 1,2,3,4,5",Shoulder Abduction [deg],0.71,0.02
4,"SimCC, Cams: 1,2,3,4,5",Shoulder Flexion [deg],0.85,0.00
5,"SimCC, Cams: 1,2,3,4,5",Trunk Displacement,0.84,0.01
10,"Metrabs, Cams: 1,2,3,4,5",Elbow Velocity [deg/s],0.86,0.00
9,"Metrabs, Cams: 1,2,3,4,5",Elbow Flexion [deg],0.96,0.00
11,"Metrabs, Cams: 1,2,3,4,5",Hand Velocity [mm/s],0.85,0.01
7,"Metrabs, Cams: 1,2,3,4,5",Shoulder Flexion [deg],0.92,0.00


In [41]:
print(df_plot.style.to_latex(caption='Pearson Correlation and corresponding P-Value',
      label = 'tab:res:cont:pearson_corr'))

\begin{table}
\caption{Pearson Correlation and corresponding P-Value}
\label{tab:res:cont:pearson_corr}
\begin{tabular}{lllrr}
 & id_s_name & metric & correlation & p_value \\
0 & SimCC, Cams: 1,2,3,4,5 & Elbow Flexion [deg] & 0.887301 & 0.048570 \\
1 & SimCC, Cams: 1,2,3,4,5 & Elbow Velocity [deg/s] & 0.779175 & 0.012742 \\
2 & SimCC, Cams: 1,2,3,4,5 & Hand Velocity [mm/s] & 0.783233 & 0.024572 \\
3 & SimCC, Cams: 1,2,3,4,5 & Shoulder Abduction [deg] & 0.711583 & 0.023773 \\
4 & SimCC, Cams: 1,2,3,4,5 & Shoulder Flexion [deg] & 0.851472 & 0.004710 \\
5 & SimCC, Cams: 1,2,3,4,5 & Trunk Displacement & 0.838084 & 0.008703 \\
10 & Metrabs, Cams: 1,2,3,4,5 & Elbow Velocity [deg/s] & 0.859563 & 0.002017 \\
9 & Metrabs, Cams: 1,2,3,4,5 & Elbow Flexion [deg] & 0.959846 & 0.000002 \\
11 & Metrabs, Cams: 1,2,3,4,5 & Hand Velocity [mm/s] & 0.849481 & 0.006643 \\
7 & Metrabs, Cams: 1,2,3,4,5 & Shoulder Flexion [deg] & 0.918850 & 0.002157 \\
6 & Metrabs, Cams: 1,2,3,4,5 & Shoulder Abduction [deg] 